## Pong 01

El siguiente programa muestra la primera versión de pong: 
Hemos definido una clae para la bola (`Ball`). La clase implementa los
mñetodos update (Por ahora solo se mueve hacia la derecha) y
`draw`, de forma que la responsabilidad de actualizar su posición
y la de pintarse en pantalla son asumidas por la propia bola:

In [11]:
import math
import pygame
import random
import time

from pygame.locals import Rect
from math import pi
from vectores import Vector2

SIZE = WIDTH, HEIGHT = 800, 640  # Tamaño de pantalla

BLACK = (0, 0, 0)                # Colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
YELLOW = (255, 255, 0)

FPS = 30                         # Velocidad del juego

class Ball:
    
    def __init__(self, x=0, y=0):
        self.pos = Vector2(x, y)
        self.speed = 10
        self.orientation = Vector2(1, 0)
        self.orientation.theta = random.uniform(pi/4, -pi/4)
        
    def update(self):
        self.pos += self.orientation * self.speed
    
    def draw(self, canvas):
        rect = pygame.Rect((self.pos.x-5, self.pos.y-5), (11, 11))
        pygame.draw.rect(canvas, WHITE, rect)

        
class Paddle:
    
    def __init__(self, x, y):
        self.pos = Vector2(x, y)
        self.rect = Rect((0, 0), (14, 100))
        self.rect.center = self.pos
        
    def update(self):
        pass
    
    def draw(self, screen):
        pygame.draw.rect(screen, WHITE, self.rect)
        
    
pygame.init()
try:
    pygame.display.set_caption("Pong 03")
    screen = pygame.display.set_mode(SIZE, 0, 24)
    limits = screen.get_rect()
    
    # Parte de inicialización del juego
    r_paddle = Paddle(WIDTH-50, HEIGHT//2)
    ball = Ball(limits.centerx, limits.centery)
    clock = pygame.time.Clock()    
    in_game = True
    while in_game:
        # Obtener datos de entrada
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                in_game = False
        # Recalcular el estado del juego, en base al estado actual y a las entradas
        ball.update()
        r_paddle.update()
        # Representamos el nuevo estado
        screen.fill(BLACK)
        ball.draw(screen)
        r_paddle.draw(screen)
        pygame.display.update()
        clock.tick(FPS)
finally:
    pygame.quit()

### Hemos implementado una clase para las raquetas

Una primera version:
    
```Python
class Paddle:

    def __init__(self, x, y):
        self.pos = Vector2(x, y)
        self.rect = Rect((0, 0), (14, 100))
        self.rect.center = self.pos

    def update(self):
        pass

    def draw(screen):
        pygame.draw.rect(screen, WHITE, self.rect)
```

Que por el momento tiene dos graves deficiencias:

- No se mueve
- La pelota no rebota cuando impacta con la raqueta

Vamos a resolverlas una por una. Empezaremos con el control.

### Eventos

Hasta ahora solon os hemos preocuado por un evento de entrada, concretamente
el evento de salida del juego (Cuando cerramos la ventana del mismo). Pygame
en realidad puede controlar muchos más eventos.

Los eventos son la manera que tiene el sistema de decirnos las cosas que pasan. 
Cada vez que se pulsa una tecla, se mueve un ratón o se cambia la inclinación
de un joystick, el sistema nos informa mediante eventos. De hecho,
cuando se pulsa una tecla, se generan **dos** eventos, uno cuando la tecla
es pulsada (evento del tipo `KEYDOWN`, como veremos) y otra cuando la tecla
se deja de pulsar (evento de tipo `KEYUP`). Esto se usa, por ejemplo,
para la autorepetición en editores de texto.

Todos los eventos tienen una característica común: tienen un atributo o campo
`type` que nos sirve para identificar el tipo de evento. Después, para cada
tipo de evento, habra unos u otros atributos que nos dan más información sobre
lo que ha pasado. Por ejemplo, para los eventos del tipo `KEYDOWN`
tenemos todos estos datos:

 - Un código numérico que indica que tecla ha sido pulsada: `key`
 - Los modificadores del teclado activos, como _Shit_, _Alt_, _Num Lock_,
   la tecla de control derecha, etc... en el atributo `mod`
 - El caracter que corresponde a la combinación de teclas pulsada, en `unicode`

> **Qué evento debo capturar, el `KEYDOWN` o el `KEYUP`:** Para este caso, nos interesa
capturar los dos. Podemos capturar el KEYDOWN de una determinada tecla que inicia
el movimiento del _paddle_ o raqueta, y cuando detectemos el `KEYUP` paramos. En
otros casos, por ejemplo un botón de disparo, simplemente usamos el teclado como
si fuera un montón de interruptores y podemos usar el que queramos.

El siguiente programa cambia los colores de la pantalla segun la tecla pulsada.
Usamos el código de las teclas `R`, `G` y `B` para cambiar el color de la pantalla,
la tecla espacio (cuyo valor está alamacenado en `K_SPACE`) vuelve a poner
la pantalla en blanco:


In [ ]:
import math
import pygame
import random
import time

from pygame.locals import Rect

SIZE = WIDTH, HEIGHT = 800, 640  # Tamaño de pantalla

BLACK = (0, 0, 0)                # Colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
YELLOW = (255, 255, 0)

FPS = 30
    
pygame.init()
try:
    pygame.display.set_caption("Keyboard Events")
    screen = pygame.display.set_mode(SIZE, 0, 24)
    
    # Parte de inicialización del juego
    clock = pygame.time.Clock()    
    in_game = True
    color = WHITE
    font = pygame.font.SysFont('Arial', size=44)
    code = ''
    while in_game:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                in_game = False
            elif event.type == pygame.KEYDOWN:
                code = str(event.key)
                if event.key == pygame.K_ESCAPE:
                    in_game = False
                elif event.key == pygame.K_g:
                    color = GREEN
                elif event.key == pygame.K_r:
                    color = RED
                elif event.key == pygame.K_b:
                    color = BLUE
                elif event.key == pygame.K_SPACE:
                    code = ''
                    color = WHITE
        screen.fill(color)
        if code:
            s = font.render(code, True, BLACK)
            screen.blit(s, (10, 10))
        pygame.display.update()
        clock.tick(FPS)
finally:
    print('Salimos del juego')
    pygame.font.quit()
    pygame.quit()

Para examinar la lista de eventos que se han producido desde la última
vez que se preguntó (normalmente el _frame_ anterior) se usa la llamada
`pygame.event.get`. Esta función nos devuelve una lista con los eventos
ordedados según el momento en que se produjeron: el último evento de la lista será
el último producido. Esto puede ser  interesante para aplicaciones que usan
el ratón para dibujar, por ejemplo.

Vamos a refactorizar un poco el código. En este caso estamos interesados (por el momento) solo en
unas cuantas cosas que pueden pasar:
    
- El jugador izquierdo mueve la raqueta hacia arriba
- El jugador izquierdo mueve la raqueta hacia abajo
- El jugador derecho mueve la raqueta hacia arriba
- El jugador derecho mueve la raqueta hacia abajo
- El juego termina (Ya sea por cerrar la ventana o por pulsar la tecla _escape_)

Vamos a hacer una clase para almacenar esta información a partir de la lista
de eventos producidos:

```Python
class InputState:
    
    def __init__(self):
        self.left_up = False
        self.left_down = False
        self.right_up = False
        self.right_down = False
        self.exit = False
        
    def update(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.exit = True
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    self.exit = True
                elif event.key == pygame.K_q:
                    self.left_up = True
                elif event.key == pygame.K_a:
                    self.left_down = True
                elif event.key == pygame.K_UP:
                    self.right_up = True
                elif event.key == pygame.K_DOWN:
                    self.right_down = True
            elif event.type == pygame.KEYUP:
                if event.key == pygame.K_q:
                    self.left_up = False
                elif event.key == pygame.K_a:
                    self.left_down = False
                elif event.key == pygame.K_UP:
                    self.right_up = False
                elif event.key == pygame.K_DOWN:
                    self.right_down = False
```

In [ ]:
El siguiente programa prueba la clase InputState:

In [ ]:
import math
import pygame
import random
import time

from pygame.locals import Rect

SIZE = WIDTH, HEIGHT = 800, 640  # Tamaño de pantalla

BLACK = (0, 0, 0)                # Colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
YELLOW = (255, 255, 0)

FPS = 30

class InputState:
    
    def __init__(self):
        self.left_up = False
        self.left_down = False
        self.right_up = False
        self.right_down = False
        self.exit = False
        
    def update(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.exit = True
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    self.exit = True
                elif event.key == pygame.K_q:
                    self.left_up = True
                elif event.key == pygame.K_a:
                    self.left_down = True
                elif event.key == pygame.K_UP:
                    self.right_up = True
                elif event.key == pygame.K_DOWN:
                    self.right_down = True
            elif event.type == pygame.KEYUP:
                if event.key == pygame.K_q:
                    self.left_up = False
                elif event.key == pygame.K_a:
                    self.left_down = False
                elif event.key == pygame.K_UP:
                    self.right_up = False
                elif event.key == pygame.K_DOWN:
                    self.right_down = False
                    

pygame.init()
try:
    pygame.display.set_caption("Keyboard Events")
    screen = pygame.display.set_mode(SIZE, 0, 24)
    
    # Parte de inicialización del juego
    clock = pygame.time.Clock()
    input_state = InputState()
    upper_left = Rect((0, 0), (50, 50))
    upper_right = Rect((WIDTH-50, 0), (WIDTH, 50))
    down_left = Rect((0, HEIGHT-50), (50, HEIGHT))
    down_right = Rect((WIDTH-50, HEIGHT-50), (WIDTH, HEIGHT))
    
    while True:
        input_state.update()
        if input_state.exit:
            break
        screen.fill(BLACK)
        if input_state.left_up:
            pygame.draw.rect(screen, RED, upper_left)
        if input_state.left_down:
            pygame.draw.rect(screen, RED, down_left)
        if input_state.right_up:
            pygame.draw.rect(screen, RED, upper_right)
        if input_state.right_down:
            pygame.draw.rect(screen, RED, down_right)
        pygame.display.update()
        clock.tick(FPS)
finally:
    print('Salimos del juego')
    pygame.quit()

Ahora tenemos que añadir a las raquetas la capacidad de moverse arriba, abajo
o de parar. Al método `update` podemos pasarle el nuevo objeto que
almacena la información del estado de entrada, para que pueda tomar
las decisiones oportunas:

```Python
class Paddle:

    def __init__(self, x, y):
        self.rect = Rect((0, 0), (14, 100))
        self.rect.center = (x, y)
        self.speed = 0

    def update(self, inputs):
        if inputs.left_up:
            self.go_up()
        elif inputs.left_down:
            self.go_down()
        else:
            self.stop()  
        self.rect.move(0, self.speed)

    def draw(screen):
        pygame.draw.rect(screen, WHITE, self.rect)
        
    def go_up(self):
        self.speed = -10

    def go_down(self):
        self.speed = 10
        
    def stop(self):
        self.speed = 0
```

In [6]:
import math
import pygame
import random
import time

from pygame.locals import Rect
from math import pi
from vectores import Vector2

SIZE = WIDTH, HEIGHT = 800, 640  # Tamaño de pantalla

BLACK = (0, 0, 0)                # Colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
YELLOW = (255, 255, 0)

FPS = 30                         # Velocidad del juego


class Paddle:

    def __init__(self, x, y):
        self.rect = Rect((0, 0), (14, 100))
        self.rect.center = (x, y)
        self.speed = 0

    def update(self, inputs):
        if inputs.left_up:
            self.go_up()
        elif inputs.left_down:
            self.go_down()
        else:
            self.stop()         
        self.rect.center.y += self.speed

    def draw(screen):
        pygame.draw.rect(screen, WHITE, self.rect)
        
    def go_up(self):
        self.speed = -10

    def go_down(self):
        self.speed = 10
        
    def stop(self):
        self.speed = 0
        
        
class Ball:
    
    def __init__(self, x=0, y=0):
        self.pos = Vector2(x, y)
        self.speed = 10
        self.orientation = Vector2(1, 0)
        self.orientation.theta = random.uniform(pi/4, -pi/4)
        
    def update(self):
        self.pos += self.orientation * self.speed
    
    def draw(self, canvas):
        rect = pygame.Rect((self.pos.x-5, self.pos.y-5), (11, 11))
        pygame.draw.rect(canvas, WHITE, rect)

        
class Paddle:
    
    def __init__(self, x, y):
        self.pos = Vector2(x, y)
        self.rect = Rect((0, 0), (14, 100))
        self.rect.center = self.pos
        
    def update(self, inputs):
        if inputs.left_up:
            self.go_up()
        elif inputs.left_down:
            self.go_down()
        else:
            self.stop()        
        self.rect = self.rect.move(0, self.speed)
        
    def draw(self, screen):
        pygame.draw.rect(screen, WHITE, self.rect)

    def go_up(self):
        self.speed = -10

    def go_down(self):
        self.speed = 10
        
    def stop(self):
        self.speed = 0

        
class InputState:
    
    def __init__(self):
        self.left_up = False
        self.left_down = False
        self.right_up = False
        self.right_down = False
        self.exit = False
        
    def update(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.exit = True
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    self.exit = True
                elif event.key == pygame.K_q:
                    self.left_up = True
                elif event.key == pygame.K_a:
                    self.left_down = True
                elif event.key == pygame.K_UP:
                    self.right_up = True
                elif event.key == pygame.K_DOWN:
                    self.right_down = True
            elif event.type == pygame.KEYUP:
                if event.key == pygame.K_q:
                    self.left_up = False
                elif event.key == pygame.K_a:
                    self.left_down = False
                elif event.key == pygame.K_UP:
                    self.right_up = False
                elif event.key == pygame.K_DOWN:
                    self.right_down = False

    
pygame.init()
try:
    pygame.display.set_caption("Pong 03")
    screen = pygame.display.set_mode(SIZE, 0, 24)
    limits = screen.get_rect()
    
    # Parte de inicialización del juego
    r_paddle = Paddle(WIDTH-50, HEIGHT//2)
    l_paddle = Paddle(50, HEIGHT//2)
    ball = Ball(limits.centerx, limits.centery)
    clock = pygame.time.Clock()    
    input_state = InputState()
    while True:
        input_state.update()
        if input_state.exit:
            break
        ball.update()    
        r_paddle.update(input_state)
        l_paddle.update(input_state)

        screen.fill(BLACK)
        for item in (ball, r_paddle, l_paddle):
            item.draw(screen)
        pygame.display.update()
        clock.tick(FPS)
finally:
    pygame.quit()

In [10]:
import math
import pygame
import random
import time

from pygame.locals import Rect
from math import pi
from vectores import Vector2

SIZE = WIDTH, HEIGHT = 800, 640  # Tamaño de pantalla

BLACK = (0, 0, 0)                # Colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
YELLOW = (255, 255, 0)

FPS = 30                         # Velocidad del juego


class Paddle:

    def __init__(self, x, y):
        self.rect = Rect((0, 0), (14, 100))
        self.rect.center = (x, y)
        self.speed = 0

    def update(self, inputs):
        if inputs.left_up:
            self.go_up()
        elif inputs.left_down:
            self.go_down()
        else:
            self.stop()         
        self.rect.center.y += self.speed

    def draw(screen):
        pygame.draw.rect(screen, WHITE, self.rect)
        
    def go_up(self):
        self.speed = -10

    def go_down(self):
        self.speed = 10
        
    def stop(self):
        self.speed = 0
        
        
class Ball:
    
    def __init__(self, x=0, y=0):
        self.pos = Vector2(x, y)
        self.speed = 10
        self.orientation = Vector2(1, 0)
        self.orientation.theta = random.uniform(pi/4, -pi/4)
        
    def update(self):
        self.pos += self.orientation * self.speed
    
    def draw(self, canvas):
        rect = pygame.Rect((self.pos.x-5, self.pos.y-5), (11, 11))
        pygame.draw.rect(canvas, WHITE, rect)

        
class Paddle:
    
    def __init__(self, x, y):
        self.rect = Rect((0, 0), (14, 100))
        self.rect.center = (x, y)
        
    def update(self):
        self.rect = self.rect.move(0, self.speed)
        
    def draw(self, screen):
        pygame.draw.rect(screen, WHITE, self.rect)

    def go_up(self):
        self.speed = -10

    def go_down(self):
        self.speed = 10
        
    def stop(self):
        self.speed = 0

class LeftPad(Paddle):
    
    def update(self, inputs):
        if inputs.left_up:
            self.go_up()
        elif inputs.left_down:
            self.go_down()
        else:
            self.stop()        
        super().update()
    
class RightPad(Paddle):
    
    def update(self, inputs):
        if inputs.right_up:
            self.go_up()
        elif inputs.right_down:
            self.go_down()
        else:
            self.stop()        
        super().update()   
        
class InputState:
    
    def __init__(self):
        self.left_up = False
        self.left_down = False
        self.right_up = False
        self.right_down = False
        self.exit = False
        
    def update(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.exit = True
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    self.exit = True
                elif event.key == pygame.K_q:
                    self.left_up = True
                elif event.key == pygame.K_a:
                    self.left_down = True
                elif event.key == pygame.K_UP:
                    self.right_up = True
                elif event.key == pygame.K_DOWN:
                    self.right_down = True
            elif event.type == pygame.KEYUP:
                if event.key == pygame.K_q:
                    self.left_up = False
                elif event.key == pygame.K_a:
                    self.left_down = False
                elif event.key == pygame.K_UP:
                    self.right_up = False
                elif event.key == pygame.K_DOWN:
                    self.right_down = False

    
pygame.init()
try:
    pygame.display.set_caption("Pong 03")
    screen = pygame.display.set_mode(SIZE, 0, 24)
    limits = screen.get_rect()
    
    # Parte de inicialización del juego
    r_paddle = RightPad(WIDTH-50, HEIGHT//2)
    l_paddle = LeftPad(50, HEIGHT//2)
    ball = Ball(limits.centerx, limits.centery)
    clock = pygame.time.Clock()    
    input_state = InputState()
    while True:
        input_state.update()
        if input_state.exit:
            break
        ball.update()    
        r_paddle.update(input_state)
        l_paddle.update(input_state)

        screen.fill(BLACK)
        for item in (ball, r_paddle, l_paddle):
            item.draw(screen)
        pygame.display.update()
        clock.tick(FPS)
finally:
    pygame.quit()